In [ ]:
import os

In [ ]:
os.environ

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}"
    # combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


In [3]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [36]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '4e079edc'}

In [4]:
n = len(documents)

for i in range(n):
    documents[i]['id'] = i 

In [9]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': 3}

In [5]:
from collections import defaultdict

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [7]:
len(hashes), len(documents)

(948, 948)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [9]:
# hashes['593f7569']
hashes['ca3dc12d']

[{'text': 'This error comes up on the Spark video 5.3.1 - First Look at Spark/PySpark,\nbecause as at the creation of the video, 2021 data was the most recent which utilised csv files but as at now its parquet.\nSo when you run the command spark.createDataFrame(df1_pandas).show(),\nYou get the Attribute error. This is caused by the pandas version 2.0.0 which seems incompatible with Spark 3.3.2, so to fix it you have to downgrade pandas to 1.5.3 using the command pip install -U pandas==1.5.3\nAnother option is adding the following after importing pandas, if one does not want to downgrade pandas version (source) :\npd.DataFrame.iteritems = pd.DataFrame.items\nNote that this problem is solved with Spark versions from 3.4.1',
  'section': 'Module 5: pyspark',
  'question': "AttributeError: 'DataFrame' object has no attribute 'iteritems'",
  'course': 'data-engineering-zoomcamp',
  'id': 'ca3dc12d'},
 {'text': 'Another alternative is to install pandas 2.0.1 (it worked well as at the time of

In [9]:
import json

In [10]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [11]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": 0
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [12]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [ ]:
!pip install -U openai

In [13]:
from openai import OpenAI
client = OpenAI()

In [14]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [15]:
from tqdm.auto import tqdm

In [16]:
results = {}

In [20]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
import pickle

In [59]:
with open('results.bin', 'wb') as file:
    pickle.dump(results, file)

print("Dane zostały zapisane do pliku 'results.bin'.")

Dane zostały zapisane do pliku 'results.bin'.


In [26]:
!wget 'https://github.com/DataTalksClub/llm-zoomcamp/raw/main/03-vector-search/eval/results.bin'

--2024-07-09 23:44:44--  https://github.com/DataTalksClub/llm-zoomcamp/raw/main/03-vector-search/eval/results.bin
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/results.bin [following]
--2024-07-09 23:44:44--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/results.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 403252 (394K) [application/octet-stream]
Saving to: ‘results.bin’

results.bin         100%[===================>] 393.80K  --.-KB/s    in 0.09s   

2024-07-09 23:44:45 (4.39 MB/s) - ‘result

In [60]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [45]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [61]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [47]:
print(json_questions)

[
    "How should I destroy infrastructure created using the CD-Deploy Action in AWS?",
    "What is the first step to destroy AWS infrastructure created via GitHub Actions?",
    "Can I destroy infrastructure created with GitHub Actions from my local setup?",
    "What command initializes Terraform with specific backend configuration before destroying infrastructure?",
    "How do you specify the variable file when running terraform destroy?"
]


In [23]:
json_questions = [
r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [24]:
json.dumps(json_questions)

'["How can I resolve the Docker error \'invalid mode: \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?", "What should I do if I encounter an invalid mode error in Docker on Windows?", "What is the correct mounting path to use in Docker for PostgreSQL data on Windows?", "Can you provide an example of a correct Docker mounting path for PostgreSQL data?", "How do I correct the mounting path error in Docker for \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?"]'

In [48]:
results[doc_id] = json.dumps(json_questions)

In [62]:
parsed_results

{'c02e79ef': ['When does the course begin?',
  'How can I get the course schedule?',
  'What is the link for course registration?',
  'How can I receive course announcements?',
  'Where do I join the Slack channel?'],
 '1f6520ca': ['Where can I find the prerequisites for this course?',
  'How do I check the prerequisites for this course?',
  'Where are the course prerequisites listed?',
  'What are the requirements for joining this course?',
  'Where is the list of prerequisites for the course?'],
 '7842b56a': ['Can I enroll in the course after it starts?',
  'Is late registration possible?',
  'Am I eligible to submit homework if I join late?',
  'Are there deadlines for final projects if I join late?',
  'Can I submit all assignments at the end of the course if I start late?'],
 '0bbf41ec': ['When will I receive the confirmation email after registering for the Data Engineering Bootcamp?',
  'Do I need the confirmation email to start the Data Engineering Bootcamp?',
  'Can I begin lea

In [63]:
documents

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 0},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': 1},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that th

In [65]:
doc_index = {d['id']: d for d in documents}

In [66]:
doc_index

{0: {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 0},
 1: {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': 1},
 2: {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however

In [67]:
if 'c02e79ef' in doc_index:
    print("Klucz istnieje w doc_index.")
else:
    print("Klucz nie istnieje w doc_index.")

Klucz nie istnieje w doc_index.


In [68]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))


KeyError: 'c02e79ef'

In [58]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv